In [ ]:
from fileinput import filename
import sys
import os
import keyboard
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from dotenv import load_dotenv
from dotenv import dotenv_values
from pathlib import Path
import logging.config
import json

logging.basicConfig(
    filename="C:\\Users\\Admin\\Documents\\Python\\Selenium\\OkCupid-Automation\\Logs.log")
LOGGER = logging.getLogger()
load_dotenv('.env')


def wait_until_clickable(driver, xpath=None, class_name=None, el_id=None, duration=10000, frequency=0.01):
    if xpath:
        WebDriverWait(driver, duration, frequency).until(
            EC.element_to_be_clickable((By.XPATH, xpath)))
    elif class_name:
        WebDriverWait(driver, duration, frequency).until(
            EC.element_to_be_clickable((By.CLASS_NAME, class_name)))
    elif el_id:
        WebDriverWait(driver, duration, frequency).until(
            EC.element_to_be_clickable((By.ID, el_id)))


def wait_until_visible(driver, xpath=None, class_name=None, el_id=None, duration=10000, frequency=0.01):
    if xpath:
        WebDriverWait(driver, duration, frequency).until(
            EC.visibility_of_element_located((By.XPATH, xpath)))
    elif class_name:
        WebDriverWait(driver, duration, frequency).until(
            EC.visibility_of_element_located((By.CLASS_NAME, class_name)))
    elif el_id:
        WebDriverWait(driver, duration, frequency).until(
            EC.visibility_of_element_located((By.ID, el_id)))


def wait_until_present(driver, xpath=None, class_name=None, el_id=None, duration=10000, frequency=0.01):
    if xpath:
        return WebDriverWait(driver, duration, frequency).until(EC.presence_of_element_located((By.XPATH, xpath)))
    elif class_name:
        return WebDriverWait(driver, duration, frequency).until(EC.presence_of_element_located((By.CLASS_NAME, class_name)))
    elif el_id:
        return WebDriverWait(driver, duration, frequency).until(EC.presence_of_element_located((By.ID, el_id)))



def input_keys(driver, xpath, keys, comment="input_keys"):
    wait_until_clickable(
        driver=driver, xpath=xpath)
    input = driver.find_element_by_xpath(xpath)
    input.clear()
    input.send_keys(keys)
    print(comment)



def input_numbers(driver, xpath, keys, comment="input_keys"):
    wait_until_clickable(
        driver=driver, xpath=xpath)
    input = driver.find_element_by_xpath(xpath)
    input.clear()
    input.send_keys(Keys.CONTROL + 'a')
    print(keys)
    input.send_keys(keys)
    print(comment)

def file_input_keys(driver, xpath, keys, comment="input_keys"):
    input = driver.find_element_by_xpath(xpath)
    input.send_keys(keys)
    print(comment)


def press_item(driver, xpath, comment="input_keys"):
    wait_until_clickable(driver=driver, xpath=xpath)
    item = driver.find_element_by_xpath(xpath)
    item.click()
    print(comment)

def press_item_present(driver, xpath, comment="input_keys"):
    wait_until_present(driver=driver, xpath=xpath)
    item = driver.find_element_by_xpath(xpath)
    item.click()
    print(comment)

def initDriver(driver):
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.set_page_load_timeout("1")

def isDisplayed(driver, xpath, comment):
    print(comment)
    try:
        driver.find_element_by_xpath(xpath)
    except EC.NoSuchElementException:
        return False
    return True

In [ ]:
def login(driver, secret, password):
    print("wait for webpage to open")
    driver.get('chrome-extension://nkbihfbeogaeaoehlefnkodbefgpgknn/home.html#initialize/create-password/import-with-seed-phrase')

    print('Close meta-mask popup')
    chwd = driver.window_handles
    driver.switch_to.window(chwd[0])
    driver.close();
    driver.switch_to.window(chwd[1])

    # Enter form Login for meta mask
    input_keys(driver, '//*[@id="app-content"]/div/div[2]/div/div/form/div[4]/div[1]/div/input', secret, comment="Input secret")
    input_keys(driver, '//*[@id="password"]', password, comment="Input Password")
    input_keys(driver, '//*[@id="confirm-password"]', password, comment="Confirm Password")
    press_item(driver, '//*[@id="app-content"]/div/div[2]/div/div/form/div[7]/div', comment="Check Box")
    press_item(driver, '//*[@id="app-content"]/div/div[2]/div/div/form/button', comment="Press Sign in")

    # SHOULD WAIT FOR LOGIN
    time.sleep(1)
    press_item(driver, '//*[@id="app-content"]/div/div[2]/div/div/button', comment="Select Continue")
    time.sleep(1)

    print("Go to opensea")
    driver.get('https://opensea.io/')
    time.sleep(1)
    press_item(driver, '//*[@id="__next"]/div/div[1]/nav/ul/div[1]/li[4]/a', comment="Select Create")
    press_item(driver, '//*[@id="main"]/div/div/div/div[2]/ul/li[1]', comment="Select Meta Mask Sign in option")
    # Wait for popup sign-in to open
    time.sleep(3)

    # Go to meta pop up page
    chwd = driver.window_handles
    driver.switch_to.window(chwd[1])
    print("Select Meta mask signin buttons")
    press_item(driver, '//*[@id="app-content"]/div/div[2]/div/div[2]/div[4]/div[2]/button[2]', comment="Select Meta Mask Sign in option")
    press_item(driver, '//*[@id="app-content"]/div/div[2]/div/div[2]/div[2]/div[2]/footer/button[2]', comment="Select Meta Mask Sign in option")
    press_item(driver, '//*[@id="app-content"]/div/div[2]/div/div[3]/button[2]', comment="Select Meta Mask Sign in option")
    time.sleep(1)

    # Go to opensea page
    chwd = driver.window_handles
    driver.switch_to.window(chwd[0])
    print("Successful Login")

def uploadContent(driver):
    f = open('json_data.json')
    data = json.load(f)
    # Loop through each item
    for i in data['nft']:
        try:
            time.sleep(1)
            file_input_keys(driver, '//*[@id="media"]', i['image'], comment="Upload image")
            input_keys(driver, '//*[@id="name"]', i['name'], comment="Enter name")
            input_keys(driver, '//*[@id="description"]', i['plot'], comment="Enter description")
            press_item(driver, '//*[@id="main"]/div/div/section/div/form/section/div[1]/div/div[2]/button', comment="Click Properties")
            time.sleep(1)

            # Place how many director inputs is needed
            for _ in range(len(i['directors']) - 1):
                time.sleep(.25)
                press_item(driver, '/html/body/div[5]/div/div/div/section/button', comment="Add Properties")
                time.sleep(.25)
            
            # Enter directors
            for index, item in enumerate(i['directors']):
                index = index + 1
                input_keys(driver, f'/html/body/div[5]/div/div/div/section/table/tbody/tr[{index}]/td[1]/div/div/input', 'director', comment="Enter description")
                input_keys(driver, f'/html/body/div[5]/div/div/div/section/table/tbody/tr[{index}]/td[2]/div/div/input', item, comment="Enter description")
               
            # Place how many genre inputs is needed
            for _ in range(len(i['genres'])):
                time.sleep(.25)
                press_item(driver, '/html/body/div[5]/div/div/div/section/button', comment="Add Properties")

            # Enter genre
            for index, item in enumerate(i['genres']):
                index = index + len(i['directors']) + 1
                input_keys(driver, f'/html/body/div[5]/div/div/div/section/table/tbody/tr[{index}]/td[1]/div/div/input', 'genre', comment="Enter genre")
                input_keys(driver, f'/html/body/div[5]/div/div/div/section/table/tbody/tr[{index}]/td[2]/div/div/input', item, comment="Enter genre")

            # Save properties
            press_item(driver, '/html/body/div[5]/div/div/div/footer/button', comment="Save Properties")

            # Add rating
            press_item(driver, '//*[@id="main"]/div/div/section/div/form/section/div[2]/div/div[2]/button', comment="Open rating")
            input_keys(driver, '/html/body/div[6]/div/div/div/section/table/tbody/tr/td[1]/div/div/input', 'Rating', comment="Enter Rating Label")
            input_numbers(driver, '/html/body/div[6]/div/div/div/section/table/tbody/tr/td[3]/div/div/input', str(10), comment="Enter Rating")
            input_numbers(driver, '/html/body/div[6]/div/div/div/section/table/tbody/tr/td[2]/div/div/input', str(9.1), comment="Enter Rating out of")
            press_item(driver, '/html/body/div[6]/div/div/div/footer/button', comment="save rating")

            print('Entered uploaded info')
            time.sleep(3)
        except ValueError:
            print(ValueError)
            exit()

        # Select block chain to use
        press_item(driver, '//*[@id="main"]/div/div/section/div/form/div[7]/div/div[2]', comment="open block chain")
        press_item(driver, '/html/body/div[1]/div/main/div/div/section/div/form/div[7]/div/div[3]/div/div/div/ul/li[1]/button', comment="select block chain")

        press_item(driver, '//*[@id="main"]/div/div/section/div/form/div[9]/div[1]/span/button', comment="submit")

        keyboard.wait('space')
        print('space was pressed, continuing to next item...')

        press_item(driver, '//*[@id="__next"]/div/div[1]/nav/ul/div[1]/li[4]/a', comment="Select Create")


# def create(driver):



def run(driver, secret=dotenv_values('.env')['SECRETKEY'], password=dotenv_values('.env')['PASSWORD']):
    maxRetries = 3
    numAttempts = 0
    retryLogin = False

    login(driver, secret, password)

    uploadContent(driver)
    # create(driver)

    # while numAttempts < maxRetries:
    #     try:
    #         login(driver, secret, password)
    #     except TimeoutException:
    #         LOGGER.info("Failed to login due to timeout. Retrying...")
    #         retryLogin = True
    #     except Exception as e:
    #         LOGGER.exception("Failed to login: " + str(e))
    #         six.reraise(Exception, e, sys.exc_info()[2])
    #     if retryLogin is False and numAttempts < maxRetries:
    #         numAttempts = maxRetries + 1
    #         login(driver, secret, password)
    #     else:
    #         exit()

In [ ]:
def chrome():
    executable_path = os.getcwd() + '\\chromedriver.exe'
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")
    options.binary_location = 'C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe'
    options.add_extension('./10.8.1_0.crx')
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(
        executable_path=executable_path, chrome_options=options)
    run(driver)


def firefox():
    executable_path = os.getcwd() + '\\geckodriver.exe'
    options = webdriver.FirefoxOptions()
    # ADD META EXTENSION
    # options.add_argument("--headless")
    driver = webdriver.Firefox(
        executable_path=executable_path, firefox_options=options)
    run(driver)


chrome()
